<a href="https://colab.research.google.com/github/chanhyeong00/machine_learning_study/blob/main/pytorch-computer%20vision/%EA%B0%9D%EC%B2%B4%20%ED%83%90%EC%A7%80/Faster%20R-CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MS COCO(Microsoft Common Objects in Context) 데이터 세트를 활용해 Faster R-CNN 모델을 미세 조정해 이미지를 분류해본다.

In [ ]:
!unzip coco.zip

MS COCO 데이터세트는 경계 상자 탐지 및 객체 분할을 위한 정보를 제공한다. 모델 학습을 위한 **데이터 주석(Data Annotation)** 정보는 annotations 디렉터리에 제공

데이터 주석 파일은 JSON 형식으로 제공되며,

정보, 라이선스, 카테고리, 이미지, 어노테이션 정보가 포함

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install cython pycocotools

# 데이터 세트

In [ ]:
import os
import torch
from PIL import Image
from pycocotools.coco import COCO
from torch.utils.data import Dataset

class COCODataset(Dataset):
    # root: 데이터 세트 경로, train:학습 데이터 불러오기 여부(False면 검증용 데이터셋)
    def __init__(self, root, train, transform=None):
        super().__init__()
        directory = "train" if train else "val"

        # 디렉터리에 있는 어노테이션 JSON 파일 경로를 설정
        annotations = os.path.join(root, "annotations", f"{directory}_annotations.json")

        # 위 경로를 pycocotools.coco.COCO 클래스에 입력
        self.coco = COCO(annotations)
        self.iamge_path = os.path.join(root, directory)
        self.transform = transform

        # 어노테이션 정보 불러오기 전에 학습에 사용되는 카테고리 정보 불러오기
        self.categories = self._get_categories() # 모델 추론 시 카테고리 정보를 확인하기 위함

        # 이미지와 어노테이션 정보를 불러온다.
        self.data = self._load_data()


    def _get_categories(self):
        categories = {0: "background"} # 0은 배경

        for category in self.coco.cats.values(): # cats 속성에서 가테고리 정보 불러오기
        # cats 속성은 딕셔너리로,
        # 상위 카테고리(supercategory), 카테고리 ID(id), 카테고리 이름(name) 정보 포함
            categories[category["id"]] = category["name"]
        return categories

    def _load_data(self):
        data = []
        # coco.imgs 속성은 어노테이션 JSON 파일의 이미지 정보(images)를 순차적으로 반환
        # 어노테이션 정보는 이미지 ID와 매핑될 수 있으므로 이미지 ID(_id) 추출
        for _id in self.coco.imgs:
            # 입력된 이미지 ID를 받아 어노테이션 정보를 반환
            # 한 번에 여러 ID를 입력받을 수 있어 리스트 형식을 반환
            # 첫 번째 어노테이션 정보를 가져와 파일 이름 추출하고
            file_name = self.coco.loadImgs(_id)[0]["file_name"]
            # 이미지 경로 불러오기
            image_path = os.path.join(self.iamge_path, file_name)
            # 이미지 불러오기
            image = Image.open(image_path).convert("RGB")

            boxes = []
            labels = []
            # 어노테이션 ID를 coco.getAnnIds에 이미지 Id 입력하면 어노테이션 ID 반환
            # 반환된 id를 coco.loadAnns로 전달해 어노테이션 정보 불러오기
            anns = self.coco.loadAnns(self.coco.getAnnIds(_id))
            for ann in anns: # 어노테이션 돌기
                x, y, w, h = ann["bbox"] # 경계 상자(bbox) 정보 추출

                boxes.append([x, y, x + w, y + h]) # 경계 상자 정보 리스트에 저장
                labels.append(ann["category_id"]) # 레이블 저장

            target = {
            "image_id": torch.LongTensor([_id]),
                "boxes": torch.FloatTensor(boxes), # 경계상자는 FloatTensor
                "labels": torch.LongTensor(labels) # label은 LongTensor
            }
            data.append([image, target])
        return data

    # 호출 메서드
    def __getitem__(self, index):
        image, target = self.data[index]
        if self.transform: # transform 변수가 있다면 이미지 변환을 적용한 이미지 반환
            image = self.transform(image)
        return image, target

    def __len__(self):
        return len(self.data)


# 데이터 로더

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader

# 데이터에 패딩 적용
def collator(batch):
    return tuple(zip(*batch))

transform = transforms.Compose(
    [
        transforms.PILToTensor(), # PIL 이미지를 Tensor 로 변환
        transforms.ConvertImageDtype(dtype=torch.float) # 텐서 이미지를 다시 float 형식으로 변환
    ]
    # Faster R-CNN이 float 형식의 0.0 ~ 1.0 범위를 갖는 이미지 텐서를 사용하기 때문이다.
    # 또한, 서로 다른 크기의 이미지를 입력받아 모델 내부에서 크기를 변경하므로 크기를 조절하지 않아도 된다.
)

train_dataset = COCODataset("coco", train=True, transform=transform)
test_dataset = COCODataset("coco", train=False, transform=transform)


# COCO 데이터세트는 이미지 내에 여러 객체 정보가 담길 수 있으므로 데이터 길이가 다를 수 있다.
# 그러므로 데이터 로더에 집합 함수(collator_fn)을 적용해 데이터를 패딩한다.
train_dataloader = DataLoader(
    train_dataset, batch_size=4, shuffle=True, drop_last=True, collate_fn=collator
)
test_dataloader = DataLoader(
    test_dataset, batch_size=1, shuffle=True, drop_last=True, collate_fn=collator
)

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


이미지의 크기를 조절하거나 대칭 등 어노테이션 정보가 달라지는 변환이 적용되면, 어노테이션 정보도 변경해야 한다.


앞서 데이터 증가 예제처럼 클래스를 재정의하거나 파이토치 깃허브에서 제공하는 객체 검출을 위한 스키립트를 사용한다. 객체 검출을 위한 스크립트는 pytorch/vision/references/detection 의 transform.py 코드를 참고



# 모델 준비

Faster R-CNN 모델의 백본으로 사용하려는 VGG-16 모델과 영역 제안 네트워크, 관심 영역 풀링을 적용한다.

## 백본 및 모델 구조 정의

In [ ]:
from torchvision import models
from torchvision import ops
from torchvision.models.detection import rpn
from torchvision.models.detection import FasterRCNN

# vgg를 백본으로(마지막 분류 계층은 제외해 특징 추출 모델로 사용)
backbone = models.vgg16(weights="VGG16_Weights.IMAGENET1K_V1").features # features가 특징추출
# 백본 모델은 출력 채널 수를 지정하는 out_channels 속성을 포함해야 한다
backbone.out_channels = 512 # vgg-16은 512 채널 반환하므로 512로 할당

# Faster R-CNN은 2-stage 객체 탐지 모델

# 1. 영역 제한 네트워크 -> 입력 이미지에서 객체 위치 후보군을 생성하는 데 사용
# 앵커 생성기(anchor_generator) 클래스는 객체 위치 후보군을 생성하는 데 사용된다.
# 입력 이미지의 각 픽셀에 대해 앵커 박스를 생성한다.
# 앵커 박스는 서로 다른 크기(sizes)와 종횡비(aspect_ratios)로 설정
# 앵커 박스에 사용되는 매개변수의 형식은 Tuple[Tuple[int]] 구조를 가져야 한다.
# **그러므로 콤마를 포함해 튜플 구조로 지정(,를 빼면 정수로 인식할 수 있음)**
anchor_generator = rpn.AnchorGenerator(
    sizes = ((32, 64, 128, 256, 512),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

# 2. 관심 영역 풀링 -> 객체 후보군을 입력으로 받아
# 후보군 내의 특징 맵 영역을 일정한 크기의 고정된 영역으로 샘플링
# 다중 스케일 관심 영역 정렬(MultiScaleRoIAlign) 클래스는
# 관심 영역 정렬(ROI Align) 기능이 포함된 클래스로 다중 스케일 이미지에서 관심 영역 풀링 진행
roi_pooler = ops.MultiScaleRoIAlign(
    featmap_names=["0"], # 특징맵 이름
    # VGG-16 모델의 특징 추출 계층은 "0"으로 정의되어 있다.

    output_size=(7,7),  # 관심 영역 풀링을 통해 추출된 특징맵 크기(H, W)
    sampling_ratio=2 # 샘플링 비율
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = FasterRCNN(
    backbone = backbone,
    num_classes=3, # 배경도 클래스에 포함되므로 클래스 개수는 3으로 ?
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
).to(device)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 69.8MB/s]


## 최적화 함수 및 학습률 스케쥴러

In [ ]:
from torch import optim

# 모델 매개변수중 학습이 가능한 매개변수만 저장해서 경사하강법 적용
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)

# 학습률 스케줄러(지정된 주기마다 학습률을 감소시킨다)
# 5 에폭(step_size)마다 학습률이 0.1(gamma) 씩 줄어든다.
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
# 스케줄러도 optimizer 변수처럼 step 메서드로 학습률을 갱신할 수 있다.
# 일반적으로 한 에폭이 완료된 후에 호출

# 모델 학습

## Faster R-CNN 미세 조정

In [ ]:
for epoch in range(5):
    cost = 0.0
    for idx, (images, targets) in enumerate(train_dataloader):
        images = list(image.to(device) for image in images)
        # 딕셔너리 간소화를 통해 적용
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # loss dict는 loss_classifier, loss_box_reg,
        # 객체유무손실(loss_objectness), 영역 제안 네트워크 손실(loss_rpn_box_reg)
        # 로 아뤄져 있다.
        # Faster R-CNN은 학습모드일 때 모든 손실값을 출력한다.
        loss_dict = model(images, targets)

        # 네개의 손실이 모두 최소가 되는 방향으로 학습해야 하므로
        # 손실값을 모두 더해 역전파를 계산한다.
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        cost += losses

    lr_scheduler.step() # 스케줄러 적용
    cost = cost / len(train_dataloader)
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

Epoch :    1, Cost : 0.444
Epoch :    2, Cost : 0.306
Epoch :    3, Cost : 0.281
Epoch :    4, Cost : 0.268
Epoch :    5, Cost : 0.255


## 모델 추론 및 시각화

In [ ]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from torchvision.transforms.functional import to_pil_image

# Pillow 라이브러리로 사각형과 텍스트를 이미지 위에 그리는 함수
def draw_bbox(ax, box, text, color):
    ax.add_patch(
        plt.Rectangle(
            xy=(box[0], box[1]),
            width=box[2] - box[0],
            height=box[3] - box[1],
            fill=False,
            edgecolor=color,
            linewidth=2,
        )
    )
    ax.annotate(
        text=text,
        xy=(box[0] - 5, box[1] - 5),
        color=color,
        weight="bold",
        fontsize=13,
    )

threshold = 0.5
categories = test_dataset.categories
with torch.no_grad():
    model.eval() # 평가모드
    for images, targets in test_dataloader:
        images = [image.to(device) for image in images]
        outputs = model(images) # 추론

        # 하나의 출력만 봄 (3개의 정보가 담겨있음)
        boxes = outputs[0]["boxes"].to("cpu").numpy()
        labels = outputs[0]["labels"].to("cpu").numpy()
        scores = outputs[0]["scores"].to("cpu").numpy()

        # 임계값보다 높은 애들을 고름
        boxes = boxes[scores >= threshold].astype(np.int32)
        labels = labels[scores >= threshold]
        scores = scores[scores >= threshold]

        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(1, 1, 1)
        plt.imshow(to_pil_image(images[0]))

        for box, label, score in zip(boxes, labels, scores):
            draw_bbox(ax, box, f"{categories[label]} - {score:.4f}", "red")

        tboxes = targets[0]["boxes"].numpy()
        tlabels = targets[0]["labels"].numpy()
        for box, label in zip(tboxes, tlabels):
            draw_bbox(ax, box, f"{categories[label]}", "blue")

        plt.show()

이 셀에서 사진이 너무 많이 출력되어 출력 제거

사진에서 개와 고양이 객체 검출 박스를 보여줌

# 모델 평가

In [ ]:
import numpy as np
from pycocotools.cocoeval import COCOeval # 코코 평가 클래스


with torch.no_grad():
    model.eval()
    coco_detections = [] # 추출 결과들
    for images, targets in test_dataloader:
        images = [img.to(device) for img in images]
        outputs = model(images)

        for i in range(len(targets)):
            image_id = targets[i]["image_id"].data.cpu().numpy().tolist()[0]
            boxes = outputs[i]["boxes"].data.cpu().numpy()
            boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
            scores = outputs[i]["scores"].data.cpu().numpy()
            labels = outputs[i]["labels"].data.cpu().numpy()

            # 임곗값을 두지 않고 모든 추출 결과를 저장
            for instance_id in range(len(boxes)):
                box = boxes[instance_id, :].tolist()

                # 추출 결과는 [이미지 id, X, Y, W, H, 점수, label] 구조로 저장
                prediction = np.array(
                    [
                        image_id,
                        box[0],
                        box[1],
                        box[2],
                        box[3],
                        float(scores[instance_id]),
                        int(labels[instance_id]),
                    ]
                )

                coco_detections.append(prediction)

    coco_detections = np.asarray(coco_detections)
    # coco dataset 실젯값 API
    coco_gt = test_dataloader.dataset.coco
    # 탐지된 결과
    # loadRes로 COCO API 형식으로 변경
    coco_dt = coco_gt.loadRes(coco_detections)
    # COCOeval클래스로 평균 정밀도 계산
    coco_evaluator = COCOeval(coco_gt, coco_dt, iouType="bbox")
    # 정밀도, 재현율 계산
    coco_evaluator.evaluate()
    # evaluate 메서드로 계산한 평균 정밀도(Average Precision, AP)와
    # 평균 재현율(Average recall) 누적
    coco_evaluator.accumulate()
    # 누적한 결과를 요약하고 출력
    coco_evaluator.summarize()

Loading and preparing results...
Converting ndarray to lists...
(954, 7)
0/954
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.262
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.634
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.017
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.238
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.290
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.316
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.407
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.407
 Average Recall     (A

출력 결과의 AP는 IoU 임곗값이 할당돼 있다. 일반적으로 IoU=0.50:0.95 인 mAP(mean AP)를 성능 평가 척도로 사용한다.

IoU=0.50인 경우를 AP50, IOU=0.75인 경우를 AP75라 한다.

첫 번쨰 결과인

Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.262

는 IoU를 0.5에서 0.95까지 0.05씩 높이면서 측정한 결과로 모든 객체 크기와 최대 100개의 객체까지 계산한 **AP값을 의미**한다.  이값은 0.262로, ***모델의 성능이 낮은 편이라는 것을 의미한다.***

두 번째 측정 결과인

Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.634

는 IoU가 0.5일 때, 모든 객체 크기와 최대 100개의 객체까지 계산한 AP 값이다. 이 값은 0.634로, 모델이 객체를 검출하는 데 있어 어느 정도의 성능을 보인다는 의미이다.


